# CENTRALESUPELEC 2021-2022 - Intelligence Artificielle pour le Véhicule Autonome

Entrainement d'un réseau de neurones de classification pour le passage des feux tricolores.

L'objet de l'étude est d'entrainer un réseau qui à partir de l'état dynamique du véhicule et de l'observation du feu tricolore devant lui va décider dans quel mode ce véhicule doit se trouver. Le concepteur a défini 5 modes. Il s'agit donc de définir à chaque pas de temps quel est le bon mode. C'est un problème de classification.

L'entrainement retenu est un entrainement supervisé. Cet entrainement va se dérouler en 4 étapes:

1) La phase de récolte des données d'entrainement, de tests et de validation. Ces données seront constituées dee entrées du réseau de neurones et de la vérité terrain (ground truth) associée à chaque vecteur d'entrée.

2) La phase de préparation des datas pour les mettre dans un format adapté à l'uotil d'entrainement de keras

3) La phase d'entrainement proprement dite

4) La pahse de validation 

Si la phase de validation donne des résultats satisfaisants, le réseau sera alors bon pour son implantation opérationnelle.


# 1) Récolte des données d'entrainement

Dans notre étude, on sait le choix de produire ces données par simulation. La simulation va simuler une voiture arrivant à une intersection protégée par un feu et son comportement sera défini par de algorithmes classiques" (cf. la présentation). On enregistre à chaque pas de temps les paramètres souhaités.

Sans simulation, il aurait fallu prendre une voiture, l'instrumenter pour être capable d'enregistrer les différents paramètres et réaliser un certain nombre de roulages réels, sur la voie publique, pour accumuler suffisament de données.

On a réalisé 3000 simulations monte-carlo (sont tirées au sort en début de simulation: la vitesse de la voiture au T0 de la simulaton, la couleur du feu à ce T0, le temps écoulé depuis que le feu est passé à cette couleur et enfin la distance à partir de laquelle la voiture verra la couleur du feu. Pour chacune de ces grandeurs un tirage aléatoire de distribution uniforme est utilisé.

La vitesse initiale est tirée entre 6.1 et 13m/s
La distance de détection du feu est tirée entre 10 et 40m.

Générer 3000 scenarios aura pris 248 secondes. Combien de temps un roulage réel aurait-il demandé pour enregistrer la même quantité de données?
(Rearque: on se limite à 3000 cas pour limiter la taille de la base à manipuler en direct durant ce cours!!! 4,4Go non compressée)

Le résultat est un fichier au format csv ou chaque ligne correspond à un pas de temps. Passage d'un cas à l'autre séquentiellement, sans mesure particulière au niveau de la transition. 

Variables stockées sur une ligne:
	la position de Ego car (en m)
	la vitesse d'ego car (en m/s) 
	l'accélératio  d'ego car (en m/s²)
	Hot1(n-1)
	Hot2(n-1)
	Hot3(n-1)
(Hot1(n-1), Hot2(n-1), Hot3(n-1)) one hot encoding de la couleur du feu au pas de temps précédent: (0,0,0): feu non détecté; (1;0;0): feu vert; (0;1;0): feu orange; (0;0;1): feu rouge

	Hot1(n)
	Hot2(n)
	Hot3(n)
(Hot1(n), Hot2(n), Hot3(n)) = one hot encoding de la couleur du feu au pas de temps courant:(0,0,0): feu non détecté; (1;0;0): feu vert; (0;1;0): feu orange; (0;0;1): feu rouge

	HotMod0(n-1)
	HotMod1(n-1)
	HotMod2(n-1)
	HotMod3(n-1)
	HotMod4(n-1)
(HotMod0(n-1); HotMod1(n-1); HotMod2(n-1); HotMod3(n-1); HotMod4(n-1)) = one hot encoding de la prediction du Mode faite au pas de temps précédent. ModePredit(n-1) = np.argmax((HotMod0(n-1); HotMod1(n-1); HotMod2(n-1); HotMod3(n-1); HotMod4(n-1))

	HotMod0GT(n)
	HotMod1GT(n)
	HotMod2GT(n)
	HotMod3GT(n)
	HotMod4GT(n)
 (HotMod0GT(n); HotMod1GT(n); HotMod2GT(n); HotMod3GT(n); HotMod4GT(n)) = one hot encoding de la vérité terrain pour l'enregistrement courant. Valeur "vérité terrain" du Mode au pas de temps présent = np.argmax((HotMod0GT(n); HotMod1GT(n); HotMod2GT(n); HotMod3GT(n); HotMod4GT(n))

Les 14 premières variables sont celles en entrée du réseau pour apprentissage; les 5 dernières = ground truth associée.

Le fichier se nomme: CS2021_CLASSIFIER_BDD_3000.txt

# 2) Préparation des datas pour l'entrainement

Il faut transférer ces données du fichier txt vers un tableau (array numpy). On passe d'abord par une liste puis on trasfère la liste dans un numpy array

In [1]:
#importations
import numpy as np
import time
from utils import load_data

# define dataset
DataSize='1'
DataName='CS2021_CLASSIFIER_BDD_'+DataSize
path = "data/"+DataName+".txt"

TestDataSize='25'
TestDataName='CS2021_CLASSIFIER_TEST_'+TestDataSize
test_path = "data/"+TestDataName+".txt"

input_size = 14
output_size = 5

InputNN, GroundTruth, LengthData = load_data(path, input_size, output_size)
TestInputNN, TestGroundTruth, TestLenghtData = load_data(test_path, input_size, output_size)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Start loading Dataset
Loading duration= 0.06981253623962402 s
Start loading Dataset
Loading duration= 0.9864249229431152 s


# 3) Phase d'entrainement - construction du modèle

Avant d'entrainer il faut construire en premier lieu le réseau. Le concepteur décide de prendre un réseau:

MLP , avec une première couche de 64 neurones avec fonction d'activation "relu", une deuxième couche de 32 neurones avec une fonction d'activation "relu", et une couche de sortie de 5 neurones (la taille du one hot encoding du mode rédit) avec fonction d'activation "softmax" (car classification).

Pour l'entrainement on prendre une loss de type "categorical_crossentropy" adaptée aux problèmes de classification avec un algorithme d'optimisation classique nommé Adam. Doc et ref: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam.

Ce modèle est construit et compilé sous Keras, une librairie pour implémenter et entrainer facilement des réseaux de neurones. Cette librairie a été incluse dans la librairie Tensorflow (développé par Google), une librairie plus complexe et complète, qui permet à la fois de créer facilement des NN, mais également d'avoir accès à des outils plus complets, comme Tensorboard que nous utiliseront pendant l'entrainement pour visualiser les performances en temps réelles.

In [8]:
#importations
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam

#Model building
model = Sequential()
model.add(Dense(64, input_dim=input_size, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(output_size, activation='softmax'))

#Model compilation
opt_adam = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt_adam , metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                960       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 165       
Total params: 3,205
Trainable params: 3,205
Non-trainable params: 0
_________________________________________________________________


# 3-2) Phase d'entrainement - Données d'entrainement vs données de test


bla bla bla

In [9]:
# training base and test base definition according to the ratio we defined
Ratio = 0.8
TrainingSize = int(LengthData*Ratio)
TestSize = LengthData - TrainingSize

# Train base extraction
#X_DNN_Train = InputNN[0:(TrainingSize-1),:]
#Y_DNN_Train = GroundTruth[0:(TrainingSize-1) , :] 

X_DNN_Train = InputNN
Y_DNN_Train = GroundTruth

# Test base extraction
#X_DNN_Test = InputNN[TrainingSize : LengthData ,:]
#Y_DNN_Test = GroundTruth[TrainingSize : LengthData , :]

X_DNN_Test = TestInputNN
Y_DNN_Test = TestGroundTruth

print(f"Train database length = {LengthData}")
print(f"Test database length = {TestLenghtData}")

Train database length = 5184
Test database length = 75019


# 3-3) Phase entrainement - entrainement


bla bla bla epoch, batch size...history

In [10]:
#classes
from tensorflow.keras.callbacks import TensorBoard
import datetime

#training process
nb_epochs= 10
batch_size = 4

time_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = f"logs/fit/{DataName}_epoch{nb_epochs}_batchsize{batch_size}_{time_date}"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

GoTrain=time.time()
model.fit(X_DNN_Train, Y_DNN_Train, epochs=nb_epochs, validation_data = (X_DNN_Test, Y_DNN_Test), batch_size=batch_size, verbose = 2, callbacks=[tensorboard_callback])

print(f"\n>>>>>>>>>>> Training completed in {(time.time() -GoTrain):.2f}s")

#Model saving
ModelName = f"{DataName}_epoch{nb_epochs}_batchsize{batch_size}_{time_date}.h5"
model.save("models/"+ModelName)
print(f"Model saved at ---> models/{ModelName}")



Epoch 1/10
1296/1296 - 12s - loss: 0.6441 - accuracy: 0.8810 - val_loss: 0.8556 - val_accuracy: 0.8979
Epoch 2/10
1296/1296 - 12s - loss: 0.0289 - accuracy: 0.9996 - val_loss: 1.0869 - val_accuracy: 0.9080
Epoch 3/10
1296/1296 - 10s - loss: 0.0075 - accuracy: 0.9996 - val_loss: 1.2585 - val_accuracy: 0.9160
Epoch 4/10
1296/1296 - 10s - loss: 0.0038 - accuracy: 0.9996 - val_loss: 1.3981 - val_accuracy: 0.9109
Epoch 5/10
1296/1296 - 11s - loss: 0.0026 - accuracy: 0.9996 - val_loss: 1.5067 - val_accuracy: 0.9122
Epoch 6/10
1296/1296 - 10s - loss: 0.0021 - accuracy: 0.9996 - val_loss: 1.5866 - val_accuracy: 0.9187
Epoch 7/10
1296/1296 - 11s - loss: 0.0019 - accuracy: 0.9996 - val_loss: 1.6740 - val_accuracy: 0.9160
Epoch 8/10
1296/1296 - 13s - loss: 0.0018 - accuracy: 0.9996 - val_loss: 1.7524 - val_accuracy: 0.9170
Epoch 9/10
1296/1296 - 12s - loss: 0.0017 - accuracy: 0.9996 - val_loss: 1.8418 - val_accuracy: 0.9135
Epoch 10/10
1296/1296 - 12s - loss: 0.0015 - accuracy: 0.9996 - val_loss:

# 3-4) Visualisation par Tensorboard

Tensorboard est un outil très utilisé et pratique pour visualiser l'avancement des apprentissage en temps réelle, pour avoir accès au comportement du réseaux de neurones au cours ou après son apprentissage.

Au cours de l'apprentissage, un fichier *log* est enregistré


In [7]:
%reload_ext tensorboard
%tensorboard --logdir="logs/fit" --host localhost

Reusing TensorBoard on port 6006 (pid 11968), started 0:01:42 ago. (Use '!kill 11968' to kill it.)

# 4) Validation

bla bla bla, on essaie sur la base de train et sur une autre base. 

In [6]:
# evaluate the model - first on the training base
from matplotlib import pyplot as plt

YTrainPredict = model.predict(X_DNN_Train)
NbErrors = 0
for index in range(TrainingSize-1):
    TrainIMode[index] = np.argmax(Y_DNN_Train [index,:])
    TrainIMode[index] = np.argmax(YTrainPredict [index,:])
    if abs(TrainIMode[index] - TrainIMode[index]) > 0.1:
        NbErrors +=1

#display training results
plt.subplot(211)
plt.title('IMode prediction error on the training base (GT ---)')
plt.plot(TrainPredictedIMode[:])
plt.plot(TrainIMode[:],'--')
plt.subplot(212)
plt.title('Prediction error ')
plt.plot(TrainPredictedIMode[:] -TrainIMode[:])
plt.show()

print('>>>>>>>>>>>>>> Number of Prediction Errors on the training base= ', NbErrors, ' out of', (TrainingSize-1),
      ' samples --> success ratio= ', (1-(NbErrors/(TrainingSize-1)))*100, '%')


ModuleNotFoundError: No module named 'matplotlib'

bla bla bla test base 

In [ ]:
### second, on the test base
YTestPredict= model.predict(X_DNN_Test)
NbErrors = 0
for index in range(TestSize):
    TestIMode[index] = np.argmax(Y_DNN_Test [index,:])
    TestPredictedIMode[index] = np.argmax(YTestPredict [index,:])
    if abs(TestIMode[index]-TestPredictedIMode[index]) >0.1:
        NbErrors +=1
        
#display results
plt.subplot(211)
plt.title('IMode prediction error on the test base (GT ---)')
plt.plot(TestPredictedIMode[:])
plt.plot(TestIMode[:],'--')
plt.subplot(212)
plt.title('Prediction error ')
plt.plot(TestPredictedIMode[:] -TestIMode[:])
plt.show()


print(f">>>>>>>>>>>>>> Number of Prediction Errors on the training base = {NbErrors} out of {(TestSize)} samples")
print(f">>>>>>>>>>>>>> Success ratio = {(1-(NbErrors/TestSize))*100:.2f}%")


bla bla bla sur une autre base de données générées selon le même processus que la base d'entrainement mais avec des tirages initiaux différents (random seed = 1999 ald 1993)

In [ ]:
# define dataset
DataSize2='25'
DataName2='CS2021_CLASSIFIER_BDD_'+DataSize2
path2 = "data/"+DataName2+".txt"

ValidInputNN, ValidGroundTruth, ValidDataLength = load_data(path2, input_size, output_size)

#Model prediction

YValidPredict = model.predict(X_DNN_Valid)
NbError = 0
for index in range(ValidDataLength-1):
    ValidIMode[index] = np.argmax(Y_DNN_Valid [index,:])
    ValidPredictedIMode[index] = np.argmax(YValidPredict [index,:])
    ValidPredError[index] = ValidIMode[index] - ValidPredictedIMode[index]
    if (abs(ValidPredError[index]) > 0.1):
        NbError +=1
        

    
#display results
plt.subplot(211)
plt.title('IMode prediction error on the Validation base (GT ---)')
plt.plot(ValidPredictedIMode[:])
plt.plot(ValidIMode[:],'--')
plt.subplot(212)
plt.title('Prediction error ')
plt.plot(ValidPredError[:])
plt.show()

print('>>>>>>>>>>>>>> Number of Prediction Errors = ', NbError, ' out of', ValidDataLength,
      ' samples --> success ratio= ', (1-(NbError/ValidDataLength))*100, '%')

# 5) Validation dans une simulation boucle fermée

blabla... on peut se tromper mais vu la dynamique des décisions vis à vis de celle de la voiture une erreur ponctuelle de prédiction peut ne pas avoir d'effet perceptible. au pire Deltax = Deltat*v 

In [11]:
from utils import Sim_TFlight

sim = Sim_TFlight(model)


In [12]:
N_rendering_runs = 5

for i in range(N_rendering_runs):
    render = True
    while render:
        sim.step()
        render = sim.render()

# 6) Discussion sur les hyperparamètres et la taille de la base de données

Au vu des résultats de la simulation en boucle fermée.

On remplit en direct un tableau Résultats simu boucle fermée = f( epochs, batch, database size)